<a href="https://colab.research.google.com/github/ashwinidverma/llm-from-scratch/blob/main/LLM_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

text = "Hello, world.  This is a test."
result =  re.split(r'([,.]|\s)',text)
print(result)
result = [item for item in result if item.strip()]
result

['Hello', ',', '', ' ', 'world', '.', '', ' ', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


['Hello', ',', 'world', '.', 'This', 'is', 'a', 'test', '.']

In [2]:
with open("/content/the-verdict.txt",'r',encoding='utf-8') as f:
  raw_text = f.read()
print("total no of character:", len(raw_text))


total no of character: 20479


In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [12]:
print(len(preprocessed))

4690


# Token IDs

In [13]:
all_words = sorted(set(preprocessed))
vocab_size= len(all_words)
#
print(vocab_size)

1130


In [14]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [16]:
vocab.items()
for i, item in enumerate(vocab.items()):
  print(item)
  if i >10:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)


In [21]:
  class SimpleTokenizerV1:
    def __init__(self,vocab):
      self.str_to_int = vocab
      self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
      preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
      preprocessed = [item.strip() for item in preprocessed if item.strip()]
      ids = [self.str_to_int[s] for s in preprocessed]
      return ids

    def decode(self, ids):
      text = " ".join([self.int_to_str[i] for i in ids])
      #replace space before the specified punctuations
      text = re.sub(r'\s+([,.?!"()\])', r'\1', text)
      return text

In [ ]:
tokenizer =